In [39]:
%pip install spylls
%pip install pagexml-tools
%pip install tqdm

4228.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


4235.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


4241.40s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [4]:
from pathlib import Path

REPOSITORY_PATH = Path().parent.absolute()
REPOSITORY_PATH

PosixPath('/Users/carstenschnober/LAHTeR/workspace/nautilusocr')

# Dictionary Lookup

In [6]:
LANGUAGE = "nl"

HUNSPELL_DICT_PATH = REPOSITORY_PATH / 'dicts'
assert HUNSPELL_DICT_PATH.is_dir()

SUFFIXES = [".aff", ".dic"]

assert HUNSPELL_DICT_PATH.is_dir()

assert all(
    (HUNSPELL_DICT_PATH / Path(LANGUAGE).with_suffix(suffix)).is_file()
    for suffix in [".aff", ".dic"]
)



In [ ]:
from spylls.hunspell import Dictionary

hunspell = Dictionary.from_files(str(HUNSPELL_DICT_PATH / Path(LANGUAGE)))

In [ ]:
def get_dict_score(tokens=["Dit", "is", "een", "text"]):

    matched_count = 0
    total_count = 0

    for token in tokens:
        total_count += len(token)

        # TODO: lowercase token?
        matched_count += hunspell.lookup(token) * len(token)

    return matched_count / total_count



In [ ]:
tokens = ["Dit", "is", "een", "text"]

get_dict_score(tokens)

0.6666666666666666

# Train Tri-Gram Model

## Functions

In [25]:
from typing import List


HYPHENS = {"-", "⸗", "="}


def get_tokens(text) -> List[str]:
    """Copied from features_epr.py"""

    tokens = list()

    new_token = ""
    for c in text:
        if c == " " and len(new_token) > 0:
            tokens.append(new_token)
            new_token = ""
        elif c == "\n" and len(new_token) > 0:
            if new_token[-1] in HYPHENS:
                new_token = new_token[:-1]
            else:
                tokens.append(new_token)
                new_token = ""
        else:
            new_token += c
    if len(new_token) > 0:
        tokens.append(new_token)

    for i, token in enumerate(tokens):
        if not token[-1].isalpha():
            tokens[i] = token[:-1]
        if not token[0].isalpha():
            tokens[i] = token[1:]

    return tokens


In [26]:
from typing import List


NGRAM_LENGTH = 3


def get_ngrams(tokens) -> List[str]:
    """Copied, adapted from features_epr.py"""

    n_grams = list()
    for token in tokens:
        token_list = list(token)
        for i in range(0, len(token_list)):
            if not token[i].isalpha():
                token_list[i] = " "
        modified_token = "".join(token_list)
        splits = modified_token.split(" ")
        for split in splits:
            if split != "":
                for i in range(0, len(split) - NGRAM_LENGTH + 1):
                    n_grams.append(split[i : i + NGRAM_LENGTH].lower())
    return n_grams


In [40]:
def get_ngram_score(ngrams, lang_ngrams) -> float:
    """Copied from features_epr.py"""
    # TODO: this is very slow
    # TODO: check if this corresponds to equation 10 in the paper?

    if len(ngrams) == 0:
        return 0

    score = 0
    for ngram in ngrams:
        for i in range(0, len(lang_ngrams)):
            if ngram == lang_ngrams[i]:
                score += 1 - (1 / len(lang_ngrams) * i)
                break

    score = score / len(ngrams)
    return score


## Read Ground Truth

In [27]:
# Download and decompress "VOC Ground truths of the trainingset in PAGE xml.7z" from https://zenodo.org/record/6414086

GROUND_TRUTH_PATH = REPOSITORY_PATH / "VOC Ground truths of the trainingset in PAGE xml"
assert (
    GROUND_TRUTH_PATH.is_dir()
), f"Directory containing ground truth not found at {GROUND_TRUTH_PATH.absolute()}"



In [78]:
import random
from collections import Counter

from pagexml.parser import parse_pagexml_file
from tqdm import tqdm

N_FILES = 20

random.seed(1)

files = list(GROUND_TRUTH_PATH.glob("*.xml"))

pagexml_files = random.sample(files, N_FILES)

lang_trigrams = list()

for file in tqdm(pagexml_files, desc="Reading PageXML files", unit="file"):
    pagexml = parse_pagexml_file(file)
    for line in pagexml.get_lines():
        tokens = get_tokens(line.text) if line.text else []
        lang_trigrams.extend(get_ngrams(tokens))

print(f"Total n-grams: {len(lang_trigrams)}")
print(f"Distinct n-grams: {len(set(lang_trigrams))}")
print(f"Most common n-grams: ")

Counter(lang_trigrams).most_common(10)


Reading PageXML files: 100%|██████████| 20/20 [00:00<00:00, 42.30file/s]

Total n-grams: 9320
Distinct n-grams: 1708
Most common n-grams: 


[('nde', 151),
 ('den', 119),
 ('der', 113),
 ('oor', 112),
 ('van', 112),
 ('ten', 92),
 ('ver', 90),
 ('eer', 81),
 ('gen', 74),
 ('een', 71)]

In [79]:
text = "Dit is eenasd teksthdgfhfgh"

tokens = get_tokens(text)
trigrams = get_ngrams(tokens)

get_ngram_score(trigrams, lang_trigrams)

0.23652762875536482

In [77]:
file = random.choice(files)
if file in pagexml_files:
    raise ValueError(f"Test file '{file}' must not be in sample of training files.")

parse_pagexml_file(file)


scores = Counter()
file_trigrams = list()

for line in pagexml.get_lines():
    text = line.text
    tokens = get_tokens(text)
    line_trigrams = get_ngrams(tokens)
    scores[text] = get_ngram_score(line_trigrams, lang_trigrams)

    file_trigrams.extend(line_trigrams)

scores.most_common()



[('teerd is voorgevallen', 0.9727282151104866),
 ('ren na dato den Resident hebben„', 0.9671975101337981),
 ('lijx vereerd soude sien wanneer', 0.9670892988474868),
 ('op soo als gesegt.', 0.9647313518504653),
 ('den tijd niets het geen eenige uijt', 0.9620598437772953),
 ('de laten weeten dat sig groote„', 0.9617285088199214),
 ('niets voorgevallen', 0.961472232940377),
 ('na de Jnlandse wijse onder een', 0.9612015595395096),
 ('ons afscheijd neemende weede„', 0.9601773029343503),
 ('Gaande den Resident met s EComp Donderdag „ 28.en', 0.9455443859875995),
 ('onderhoorige om onder een diver„', 0.943244124032783),
 ('makte sig des weegen heeden na„', 0.9401264678272988),
 ('trent 8 uuren in den avond', 0.9382421202365802),
 ('wierden onthaalt als wanneer', 0.9373009966133642),
 ('sijn komst maar ook diverse hee„', 0.931753280783752),
 ('Sondag „ 24„en', 0.9278748970577226),
 ('rom vertrocken terwijl in al', 0.9237745858244473),
 ('hem op sijn bodem quam besoeken', 0.920680969849945),
 (

In [73]:
get_ngram_score(file_trigrams, lang_trigrams)

0.9127672833674267